In [90]:
import pandas as pd

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

nlme = importr('nlme')
stats = importr("stats")
base = importr("base")

PARAMS_MAPPING = {
    "(Intercept)": "$intercepto$",
    "Sexo": "$sexo_i$",
    "Edad": "$edad_i$",
    "Mes": "$mes_j$",
    "Adherencia": "$adherencia_{ij}$",
    "Mes:Adherencia": "$mes_j\ adherencia_{ij}$",
    "Adherencia:Mes": "$mes_j\ adherencia_{ij}$",
    "Adherencia_Perfecta": "$adherencia\ perfecta_i$",
    "Mes:Adherencia_Perfecta": "$mes_j\ adherencia\ perfecta_i$",
    "Adherencia_Perfecta:Mes": "$mes_j\ adherencia\ perfecta_i$",
    "Adherencia_Total": "$\overline{adherencia}_i$",
    "Mes:Adherencia_Total": "$mes_j\ \overline{adherencia}_i$",
    "Adherencia_Total:Mes": "$mes_j\ \overline{adherencia}_i$",
    "Adherencia_lag1": "$adherencia_{ij-1}$",
    "Mes:Adherencia_lag1": "$mes_j\ adherencia_{ij-1}$",
    "Adherencia_lag1:Mes": "$mes_j\ adherencia_{ij-1}$",
    "Adherencia_Acumulada": "$\overline{adherencia}_{ij}$",
    "Mes:Adherencia_Acumulada": "$mes_j\ \overline{adherencia}_{ij}$",
    "Adherencia_Acumulada:Mes": "$mes_j\ \overline{adherencia}_{ij}$",
}

In [91]:
def add_minor_if_zero(num):
    return num if num != "0.0" else "<0.001"

def model_to_latex(model) -> str:
    model_summary = base.summary(model)
    cov_names = model_summary.rx2("coefficients").rx2("fixed").names
    n_covs = len(cov_names)
    fe_params = dict(zip(cov_names, model_summary.rx2("tTable")[:n_covs]))
    se_params = dict(zip(cov_names, model_summary.rx2("tTable")[n_covs:n_covs*2]))
    tvalues = dict(zip(cov_names, model_summary.rx2("tTable")[n_covs*3:n_covs*4]))
    pvalues = dict(zip(cov_names, model_summary.rx2("tTable")[n_covs*4:n_covs*5]))
    pvalues = {key: f"${add_minor_if_zero(str(round(value, 3)))}$" for key, value in pvalues.items()}
    table = ""
    for name in cov_names:
        table += f"\n\t    {PARAMS_MAPPING[name]} & {round(fe_params[name], 3)} & {round(se_params[name], 3)} & {round(tvalues[name], 3)} & ".replace(".", ",") + f"{pvalues[name]} \\\\"
    return fr"""
    \begin{{tabular}}{{*{{5}}{{|c}}|}}
        \hline
        \multicolumn{{3}}{{|c}}{{Log-Likelihood}} & \multicolumn{{2}}{{|c|}}{{{round(model_summary.rx2("logLik")[0], 2)}}} \\
        \multicolumn{{3}}{{|c}}{{AIC}} & \multicolumn{{2}}{{|c|}}{{{round(model_summary.rx2("AIC")[0], 2)}}} \\
        \multicolumn{{3}}{{|c}}{{BIC}} & \multicolumn{{2}}{{|c|}}{{{round(model_summary.rx2("BIC")[0], 2)}}} \\
        \hline
        Covariable				   & Coef.                         & Std. Err.                  & z                           & $P<|z|$  \\
        \hline{table}
        \hline
    \end{{tabular}}
"""

In [92]:
tesis = pd.read_csv("../Datos/tesis_final_preprocesado.csv")
tesis.head()

,idPaciente,Mes,TAS,Adherencia,Sexo,Edad,tas_basal,Adherencia_Acumulada,Adherencia_Total,Adherencia_Perfecta,...,Intercept,Adherencia_lag1,TAS_lag1,Adherencia_Acumulada_lag2,TAS_Media_Acumulada_lag2,Mes*Adherencia,Mes*Adherencia_Perfecta,Mes*Adherencia_Total,Mes*Adherencia_lag1,Mes*Adherencia_Acumulada
0,4026,1,119.000000,1,0,76.0,116,1.0,1.0,1,...,1,0.0,116.00000,0.0,116.000000,1,1,1.0,0.0,1.0
1,4026,2,127.000000,1,0,76.0,116,1.0,1.0,1,...,1,1.0,119.00000,0.0,116.000000,2,2,2.0,2.0,2.0
2,4026,3,140.000000,1,0,76.0,116,1.0,1.0,1,...,1,1.0,127.00000,1.0,119.000000,3,3,3.0,3.0,3.0
3,4026,4,146.712710,1,0,76.0,116,1.0,1.0,1,...,1,1.0,140.00000,1.0,123.000000,4,4,4.0,4.0,4.0
4,4026,5,177.708084,1,0,76.0,116,1.0,1.0,1,...,1,1.0,146.71271,1.0,128.666667,5,5,5.0,5.0,5.0


In [93]:
with (ro.default_converter + pandas2ri.converter).context():
    rdf = ro.conversion.get_conversion().py2rpy(tesis)

In [94]:
formula = ro.Formula("TAS ~ Sexo + Edad + Adherencia + Mes + Mes:Adherencia")

# Ordenada y pendiente aleatoria

In [95]:
random_formula = ro.Formula("~ 1 + Mes | idPaciente")
model_both = nlme.lme(formula, random=random_formula, data=rdf)

In [ ]:
with open("../Tesina/Mio/tables_anexo/modelo_both.tex", "w") as f:
    f.write(model_to_latex(model_both))

# Pendiente aleatoria

In [ ]:
random_formula = ro.Formula("~ Mes | idPaciente")
model_pendiente = nlme.lme(formula, random=random_formula, data=rdf)

In [ ]:
with open("../Tesina/Mio/tables_anexo/modelo_pendiente.tex", "w") as f:
    f.write(model_to_latex(model_pendiente))

# Ordenada aleatoria

In [ ]:
random_formula = ro.Formula("~ 1 | idPaciente")
model_ordenada = nlme.lme(formula, random=random_formula, data=rdf)

In [ ]:
with open("../Tesina/Mio/tables_anexo/modelo_ordenada.tex", "w") as f:
    f.write(model_to_latex(model_ordenada))

# Pendiente y ordenada aleatoria con AR1

In [ ]:
random_formula = ro.Formula("~ 1 + Mes | idPaciente")
model_both_ar1 = nlme.lme(formula, random=random_formula, data=rdf, corr=nlme.corAR1())

In [ ]:
with open("../Tesina/Mio/tables_anexo/modelo_both_ar1.tex", "w") as f:
    f.write(model_to_latex(model_both_ar1))

# Ordenada aleatoria con AR1

In [ ]:
random_formula = ro.Formula("~ 1 | idPaciente")
model_ar1_only_intercept = nlme.lme(formula, random=random_formula, data=rdf, corr=nlme.corAR1())

In [ ]:
with open("../Tesina/Mio/tables_anexo/modelo_ordenada_ar1.tex", "w") as f:
    f.write(model_to_latex(model_ar1_only_intercept))